In [1]:
# basic imports for web-scraping
# note to self: use Python 3.9.0

from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from requests import Timeout
import time, pytube

In [2]:
def youtube_scrape(search_query, num_results): # retrieving results from youtube
  driver = webdriver.Chrome(r'\\chromedriver')
  results_links = []

  link = "https://www.youtube.com/results?search_query=" + search_query
  driver.get(link)

  start_time = time.time()
  delta_time = time.time() - start_time

  while delta_time < 5: #loop loads elements for 5 seconds or until last result is reached (edge case)
    end_result = driver.find_element(By.CSS_SELECTOR, '#message').is_displayed()
    driver.execute_script("var scrollingElement = (document.scrollingElement || document.body);scrollingElement.scrollTop = scrollingElement.scrollHeight;")
    # print(end_result)
    delta_time = time.time() - start_time
    print("DELTA TIME: " + str(delta_time))
    if (end_result): 
      break
  
  print("Extracting results...")
  i = 1
  for result in driver.find_elements(By.CSS_SELECTOR, '.text-wrapper.style-scope.ytd-video-renderer'):
    link = result.find_element(By.CSS_SELECTOR, '.title-and-badge.style-scope.ytd-video-renderer a').get_attribute('href')
    #label = result.find_element(By.CSS_SELECTOR, '.title-and-badge.style-scope.ytd-video-renderer a').get_attribute('aria-label')

    results_links.append(link)
    i += 1
    if (i > num_results):
      break

  if (len(results_links) > 0):
    print(results_links[0])
  print("LENGTH OF RESULTS: " + str(len(results_links)))

  return results_links

In [3]:
# searching youtube for the listed queries and taking weighted amounts of each (takes approx. 3 mins)

search_queries = ["running form", "bad running form example", "running form mistakes posetv", "running overstriding example", "gait analysis", 
                    "heel-strike running example", "poor running form", "marathon highlights", "cross country running", "5000m race", "10000m race", "kenya track workout", 
                    "paul chelimo", "timothy cheruiyot", "long distance running luis orta", "kitata running form", "elite long run", "sweat elite training sessions"]
search_weights = [10, 20, 10, 10, 10, 10, 20, 10, 40, 20, 20, 20, 20, 20, 20, 5, 10, 20]
res = [] # list holding all results

input_file = open(".\\Web_Scrape_Precut\\web-scraped-links.txt", "r")
first_char = input_file.read(1)

if not first_char:  # if not yet scraped, scrape the web for yt links
    for i in range(len(search_queries)):
        temp_res = youtube_scrape(search_queries[i], search_weights[i])
        for j in range(len(temp_res)):
            res.append(temp_res[j])

    print("\n\nFINAL RESULTS\n\n")
    for j in range(len(res)):
        print(res[j])
        print("\n")

else:               # already scraped, read links from file
    input_file.seek(0)
    res.clear()
    res = input_file.read().split('\n')

In [ ]:
# downloading the videos from youtube

def videoName(i):
    ret = str(i)
    while (len(ret) < 4):
        ret = "0" + ret
    return "WS-NL-NC-" + ret # web scraped-no label-no cut-example number

counter = 0
already_downloaded = -1

for url in res:
    if (counter <= already_downloaded):
        counter += 1 
        continue
    try:
        #with Timeout(600):
        print('p1')
        youtube = pytube.YouTube(url)
        print (url)
        vid = youtube.streams.filter(file_extension='mp4').filter(resolution='360p').first()
        print('p3')
        vid.download(output_path=".\\Web_Scrape_Precut", filename=(videoName(counter) + ".mp4"))
        print(videoName(counter) + ' downloaded')
    except:
        print("\nFAILED TO DOWNLOAD " + str(videoName(counter)) + "\n")
    counter += 1
